In [1]:
import os,sys
sys.path.append('../src')

import MSSP
from MSSP.exceptions import MsspError

In [2]:
import uuid

In [3]:
import copy

In [4]:
gdir = '/Users/brandon/Documents/GitHub/SNAP Fisheries Spreadsheets/json/current'
outdir = '/Users/brandon/Documents/GitHub/SNAP Fisheries Spreadsheets/json/2016-06-30'

In [5]:
Eq = MSSP.MsspFromJson(gdir)

### Add N/A answers

In [6]:
help(Eq.refactor_answers)

Help on method refactor_answers in module MSSP.mssp_data_store:

refactor_answers(self, question, answers) method of MSSP.importers.from_json.JsonImporter instance
    Re-arrange or otherwise re-define the answers to a given question.  The input should include a single
    QuestionID, and a new list of the question's answers.  This list must be a SUPERSET of the question's
    existing answers [see merge_answers and delete_answer [TODO] for an antidote] and must be supplied in the
    new correct order.
    
    The method will generate a mapping from the current answers to the new answers, and then will go through
    all the criteria and caveats, re-mapping the index values.
    
    This method is made atomic by performing the mapping onto new data frames that are only installed if
    the operation is successful.
    :param question:
    :param answers:
    :return:



In [7]:
Eq.search('M10')

searchAttributes(attributes=[UUID('2157c264-a1ed-3559-ae32-e1fd5044d055')], questions=[154], targets=[])

In [8]:
Eq.questions(154)[0].valid_answers

['No', 'Yes']

In [9]:
Eq.refactor_answers(154, ['N/A', 'No', 'Yes'])

Remapping answers with mapping [1, 2]
map from: ['No', 'Yes']


In [10]:
Eq.search('M33')

searchAttributes(attributes=[UUID('d363b9c1-f764-30c0-b70f-66941e816c5b')], questions=[187], targets=[])

In [11]:
Eq.questions(187)[0].valid_answers

['No', 'Yes']

In [12]:
Eq.refactor_answers(187, ['N/A', 'No', 'Yes'])

Remapping answers with mapping [1, 2]
map from: ['No', 'Yes']


### Question 80 - merge answers

In [13]:
Eq.question(80)

Question ID: 80
"If handled or captured and released, is survivorship compromised? (AND change answer to "Yes/No" rather than "high/low")"
(Category: Biophysical/life history)
  * IF LOW
  * IF YES
  * Is species susceptible to barotrauma, and/or has low survivorship on release?
  * NEW
  * Biology/life history
  * If handled or captured and released, is survivorship compromised? 
  * R7
  * reword to "High survicorship on release?"
  * M36
  * ·       Disease/barotrauma:  Reword with less jargon.  If you catch and release, will it survive?
 References: ControlRules Row 15, ControlRules Row 22, Monitoring Col AO
Valid Answers: ['No', 'Yes', u'LOW', u'HIGH']
 Satisfied By: None
 Satisfies: None
Has Caveats:
     QuestionID  TargetID AnswerValue  \
412          80        50         Yes   
413          80        51         Yes   
414          80        52         Yes   
415          80        53         Yes   
416          80        54         Yes   
417          80        55         Yes 

In [14]:
Eq.merge_answers(80,['Yes', 'LOW'], merge_to='Yes')

Merging answers into Yes:
 Yes
 LOW
Really continue? [y]: 
Remapping answers with mapping [0, 1, 1, 3]
map from: ['No', 'Yes', u'LOW', u'HIGH']
Remapping answers with mapping [0, 1, -1, 2]
map from: ['No', 'Yes', u'LOW', u'HIGH']


In [15]:
Eq._questions[80].valid_answers

['No', 'Yes', u'HIGH']

In [16]:
Eq.merge_answers(80,['No', 'HIGH'], merge_to='No')

Merging answers into No:
 No
 HIGH
Really continue? [y]: 
Remapping answers with mapping [0, 1, 0]
map from: ['No', 'Yes', u'HIGH']
Remapping answers with mapping [0, 1, -1]
map from: ['No', 'Yes', u'HIGH']


In [17]:
Eq._questions[80].valid_answers

['No', 'Yes']

### Question 97 - make composite of 98-106

In [18]:
Eq._questions[97].satisfied_by = set(range(98,107))

In [19]:
Eq._set_satisfies(97)

In [20]:
Eq.question(97)

Question ID: 97
  * Does assessment suggest a certain form of decision rule? (PENDING OUTCOMES OF ASSESSMENT COMPONENT OF MSSP - 2nd layer?)
  * IF YES
 References: ControlRules Row 32
Valid Answers: ['No', 'Yes']
 Satisfied By: [98, 99, 100, 101, 102, 103, 104, 105, 106]
 Satisfies: None
Has Caveats:
     QuestionID  TargetID AnswerValue  \
787          97        50         Yes   
788          97        51         Yes   
789          97        52         Yes   
790          97        53         Yes   
791          97        54         Yes   
792          97        55         Yes   
793          97        58         Yes   
794          97        59         Yes   
795          97        60         Yes   
796          97        61         Yes   
797          97        64         Yes   
798          97        88         Yes   

                                                 Note  
787                            YES - if catch-related  
788                            YES - if catch-relat

### Question 110 - add title

In [21]:
Eq._attributes['88bf52a1-cb69-3820-848f-3a3cdfbca583'].text = 'Is there latent effort in the fishery?'

In [22]:
Eq.attributes(Eq.attributes_for(110))

AttributeID ec11bea9-cbd3-3736-a89d-bbc7403e6910: IF YES
AttributeID 2eb34970-1ca5-340c-b602-fde28fc7a841: Operational
AttributeID 88bf52a1-cb69-3820-848f-3a3cdfbca583: Is there latent effort in the fishery?
AttributeID 7b7dfde6-41b3-3745-80ad-a433ada48fde: R35


### Question 120 - make into two questions
put the new one at QID 130, which is blank

This is tricky- need to:
 1. clone question into Eq._questions
 2. remap attributes by visual inspection
 3. except for category- duplicate
 4. create new title for new question
 5. remap all criteria with Question ID 120 to Question ID 130 (everything splits easily criteria vs caveats)

In [23]:
Eq._questions[120].__dict__

{'_external_answers': [u'low', u'1', u'2', u'moderate', u'3', u'4', u'high'],
 '_yes_no_included': False,
 'category': UUID('ceb4663a-e214-3c99-a3f4-1e49fedb3763'),
 'references': [(u'ControlRules', (59, None)),
  (u'Monitoring', (None, 8)),
  (u'ControlRules', (60, None)),
  (u'ControlRules', (61, None))],
 'satisfied_by': set(),
 'satisfies': set(),
 'title': UUID('c661691a-999d-3a16-87bc-19506b938659'),
 'valid_answers': [u'low', u'1', u'2', u'moderate', u'3', u'4', u'high']}

In [24]:
q120 = Eq._questions[120]

In [25]:
q130 = MSSP.mssp_objects.MsspQuestion()

In [26]:
q130._external_answers = copy.copy(q120._external_answers)
q130.category = copy.copy(q120.category)
q130.references = [('Monitoring', (None,8))]
q130._update_valid_answers()

In [27]:
q130.title = Eq._attributes.add_element('How culturally ingrained is cooperation to management?')

In [28]:
Eq._questions[130] = q130

In [29]:
Eq.add_attribute_mapping(130, q130.category)

In [30]:
Eq.add_attribute_mapping(130, q130.title)

In [31]:
q120.references = [q120.references[i] for i in (0, 2, 3)]

In [32]:
q120.__dict__

{'_external_answers': [u'low', u'1', u'2', u'moderate', u'3', u'4', u'high'],
 '_yes_no_included': False,
 'category': UUID('ceb4663a-e214-3c99-a3f4-1e49fedb3763'),
 'references': [(u'ControlRules', (59, None)),
  (u'ControlRules', (60, None)),
  (u'ControlRules', (61, None))],
 'satisfied_by': set(),
 'satisfies': set(),
 'title': UUID('c661691a-999d-3a16-87bc-19506b938659'),
 'valid_answers': [u'low', u'1', u'2', u'moderate', u'3', u'4', u'high']}

In [33]:
Eq.attributes(Eq.attributes_for(120))

AttributeID 57675259-e2bb-396f-9928-3a1d9d9d9f52: Level of fisher cooperation (includes cultural preferences)
AttributeID 6d76686b-76ba-31b8-97ba-3968dada27ce: low
AttributeID 80523536-69fa-35b5-99bb-11a15baa1ffc: Socio-economic
AttributeID 5363bef5-825a-34c1-9e1c-f70551a5c2d9: CRITERIA: begin here. These will provide a first-cut shortlist of monitoring options based on the MINIMUM requirements for each form of monitoring.
AttributeID 3fc6b453-876e-3792-9187-a562235b3754: criteria
AttributeID fb85634b-9d29-3369-b134-0b0215ba9e7d: Fishery cooperation (wrt HS approach)
AttributeID b08347fc-d4fd-38d6-b9b4-a4a6c36c1cac: How culturally ingrained is cooperation to management? Categories 1) unwilling to share and unwilling to record, 2) willing to share info but unwilling to record info, 3) willing to share and record but may not do so reliably, 4) willing and reliable sharing and recording
AttributeID e4a57dc2-b4de-30b8-b466-dbf83459dfe8: moderate
AttributeID 917fe653-f24a-3c8a-9454-70ecf153

In [34]:
def move_attribute(a_id, from_q, to_q):
    try:
        Eq.add_attribute_mapping(to_q, uuid.UUID(a_id))
    except MsspError:
        pass
    Eq.del_attribute_mapping(from_q, uuid.UUID(a_id))

In [35]:
# make list of monitoring attributes that should be removed from CR question and added to M question
for x in ['4ea136c3-94aa-3dda-85b9-ab3c3d07f8ca', '3fc6b453-876e-3792-9187-a562235b3754', 
          'fb85634b-9d29-3369-b134-0b0215ba9e7d', 'b08347fc-d4fd-38d6-b9b4-a4a6c36c1cac', '5363bef5-825a-34c1-9e1c-f70551a5c2d9']:
    move_attribute(x, 120, 130)

Removed 1 reference
Removed 1 reference
Removed 1 reference
Removed 1 reference
Removed 1 reference


In [36]:
Eq.attributes(Eq.attributes_for(130))

AttributeID ceb4663a-e214-3c99-a3f4-1e49fedb3763: General
AttributeID 2e1e4837-a9d2-312e-b9f3-eb323556eef2: How culturally ingrained is cooperation to management?
AttributeID 4ea136c3-94aa-3dda-85b9-ab3c3d07f8ca: M3
AttributeID 3fc6b453-876e-3792-9187-a562235b3754: criteria
AttributeID fb85634b-9d29-3369-b134-0b0215ba9e7d: Fishery cooperation (wrt HS approach)
AttributeID b08347fc-d4fd-38d6-b9b4-a4a6c36c1cac: How culturally ingrained is cooperation to management? Categories 1) unwilling to share and unwilling to record, 2) willing to share info but unwilling to record info, 3) willing to share and record but may not do so reliably, 4) willing and reliable sharing and recording
AttributeID 5363bef5-825a-34c1-9e1c-f70551a5c2d9: CRITERIA: begin here. These will provide a first-cut shortlist of monitoring options based on the MINIMUM requirements for each form of monitoring.


In [37]:
Eq._criteria.loc[Eq._criteria['QuestionID']==120, 'QuestionID'] = 130

In [38]:
Eq.delete_answer(120, '1')
Eq.delete_answer(120, '2')
Eq.delete_answer(120, '3')
Eq.delete_answer(120, '4')
Eq.delete_answer(130, 'low')
Eq.delete_answer(130, 'moderate')
Eq.delete_answer(130, 'high')

Remapping answers with mapping [0, -1, 1, 2, 3, 4, 5]
map from: [u'low', u'1', u'2', u'moderate', u'3', u'4', u'high']
Remapping answers with mapping [0, -1, 1, 2, 3, 4]
map from: [u'low', u'2', u'moderate', u'3', u'4', u'high']
Remapping answers with mapping [0, 1, -1, 2, 3]
map from: [u'low', u'moderate', u'3', u'4', u'high']
Remapping answers with mapping [0, 1, -1, 2]
map from: [u'low', u'moderate', u'4', u'high']
Remapping answers with mapping [-1, 0, 1, 2, 3, 4, 5]
map from: [u'low', u'1', u'2', u'moderate', u'3', u'4', u'high']
Remapping answers with mapping [0, 1, -1, 2, 3, 4]
map from: [u'1', u'2', u'moderate', u'3', u'4', u'high']
Remapping answers with mapping [0, 1, 2, 3, -1]
map from: [u'1', u'2', u'3', u'4', u'high']


In [39]:
Eq.question(120)

Question ID: 120
"What is the level of fishery cooperation, in terms of complying with and supporting management measures?"
(Category: General)
  * Level of fisher cooperation (includes cultural preferences)
  * low
  * Socio-economic
  * moderate
  * high
  * R44
  * includes cultural preferences
  * How culturally ingrained is cooperation to management?
  * willing if paid
  * Are there enabling conditions
 References: ControlRules Row 59, ControlRules Row 60, ControlRules Row 61
Valid Answers: [u'low', u'moderate', u'high']
 Satisfied By: None
 Satisfies: None
Has Caveats:
      QuestionID  TargetID AnswerValue                                   Note
1089         120        50         low                                     NO
1090         120        50    moderate                                CAUTION
1091         120        50        high                                    YES
1092         120        51         low                                     NO
1093         120        51 

In [40]:
Eq.question(130)

Question ID: 130
"How culturally ingrained is cooperation to management?"
(Category: General)
  * M3
  * criteria
  * Fishery cooperation (wrt HS approach)
  * How culturally ingrained is cooperation to management? Categories 1) unwilling to share and unwilling to record, 2) willing to share info but unwilling to record info, 3) willing to share and record but may not do so reliably, 4) willing and reliable sharing and recording
  * CRITERIA: begin here. These will provide a first-cut shortlist of monitoring options based on the MINIMUM requirements for each form of monitoring.
 References: Monitoring Col H
Valid Answers: [u'1', u'2', u'3', u'4']
 Satisfied By: None
 Satisfies: None
Has Criteria:
     QuestionID  TargetID AnswerValue
264         130        98           1
265         130        99           1
266         130       100           1
267         130       101           1
268         130       102           2
269         130       103           2
270         130       104   

### Question 125: merge answers

In [8]:
#Eq.merge_answers(125, ['Yes', 'open access'], merge_to='open access')
# strange stuff going on here- answers were wrongly mapped- fixing manually

In [41]:
Eq._questions[125].valid_answers = ['intermediate', ' open access', 'x', 'limited entry', 'y']

In [42]:
Eq.delete_answer(125, 'x')

Remapping answers with mapping [0, 1, -1, 2, 3]
map from: ['intermediate', ' open access', 'x', 'limited entry', 'y']


In [43]:
Eq.delete_answer(125, 'y')

Remapping answers with mapping [0, 1, 2, -1]
map from: ['intermediate', ' open access', 'limited entry', 'y']


In [44]:
Eq.reorder_answers(125, [2, 0, 1])

Remapping answers with mapping [1, 2, 0]
map from: ['intermediate', ' open access', 'limited entry']


In [45]:
Eq.question(125)

Question ID: 125
"Is the fishery open access, as opposed to limited entry?"
(Category: General)
  * intermediate
  * Governance
  * Open access governance, to limited entry. (e.g.in an open access governance system the efficacy of tools like size , fishing season, or effort limits is often reduced by effort creep, but when access is limited this seems to be less of a problem and so the same tool would look more attractive)
  * open access
  * limited entry
  * Is the fishery open access? (as opposed to limited entry)
  * IF YES
  * CAVEATS - Second, refine the shortlist of monitoring options according to the below range of criteria
  * Strength of governance
  * R48
  * Check consistency in monitoring spreadsheet, question M8. In open access governance system, the efficacy of tools like size, fishng season, or effort limits is often reduced by effort creep, but when access is limited this seems to be less of a problem nd so the same tool would look more attractive.
  * M8
  * as oppose

### Question 152 - clone caveats across several answers

In [46]:
Eq.question(152)

Question ID: 152
"Where is the capacity and mandate to faciliate or allow for monitoring?"
(Category: Strength of governance)
  * IF REGIONAL, STATE OR NATIONAL
  * CAVEATS - Second, refine the shortlist of monitoring options according to the below range of criteria
  * What is the dominant system of governance? (Community level, Regional, State, National) (SEE ALSO COLUMN AT)
  * What is the dominant system of governance? (Community level, Regional, State, National)
  * IF COMMUNITY
  * FROM HERE AND TO THE RIGHT ARE "EXPANDED" CAVEAT CATEGORIES
  * M7
  * in this instance state and national
  * Replace with "Where is the capacity and mandate to faciliate or allow for monitoring?"
 References: Monitoring Col AP, Monitoring Col L
Valid Answers: [u'Regional, State, or National', u'IF COMMUNITY']
 Satisfied By: None
 Satisfies: None
Has Caveats:
      QuestionID  TargetID                   AnswerValue  \
1622         152        98  Regional, State, or National   
1623         152        

In [47]:
# first, reword and reorder answers
Eq._questions[152].valid_answers = ['Regional', 'Community']
Eq.reorder_answers(152, [1,0])

Remapping answers with mapping [1, 0]
map from: ['Regional', 'Community']


In [48]:
Eq.refactor_answers(152, ['Community', 'Regional', 'State', 'National'])

Remapping answers with mapping [0, 1]
map from: ['Community', 'Regional']


In [49]:
cavs = Eq._caveats.copy()
for index,row in Eq._caveats.loc[(Eq._caveats['QuestionID']==152) & (Eq._caveats['Answer'] == 1)].iterrows():
    r1 = row.copy()
    r1['Answer'] = 2
    cavs = cavs.append(r1, ignore_index=True)
    r2 = row.copy()
    r2['Answer'] = 3
    cavs = cavs.append(r2, ignore_index=True)

In [50]:
Eq._caveats = cavs

### Question 159 - keep as is

In [51]:
Eq.question(159)

Question ID: 159
"Are regulations effectively enforced?"
(Category: Enforcement capability)
  * Are regulations enforced, AND, IF they are enforced, are regulations/governance trusted and respected?  (SEE ALSO COLUMNS AY,AZ)
  * CAVEATS - Second, refine the shortlist of monitoring options according to the below range of criteria
  * IF ENFORCED, BUT NOT RESPECTED
  * Are regulations enforced, AND, IF they are enforced, are regulations/governance trusted and respected? 
  * IF NOT ENFORCED
  * IF ENFORCED, AND RESPECTED
  * M15
  * Reword to "Does enforcement capacity exist, and, if so..?"
  * ·       Regulations enforced:  Steven answered yes, but it is not effective.
 References: Monitoring Col AU, Monitoring Col AV, Monitoring Col T
Valid Answers: [u'IF NOT ENFORCED', u'IF ENFORCED, BUT NOT RESPECTED', u'IF ENFORCED, AND RESPECTED']
 Satisfied By: None
 Satisfies: None
Has Caveats:
      QuestionID  TargetID                     AnswerValue  \
1931         159       102               

### Question 168 - keep yes/no

In [52]:
Eq.question(168)

Question ID: 168
"Is there illegal, unregulated, or unreported fishing?"
(Category: Operational characteristics)
  * IF YES
  * CAVEATS - Second, refine the shortlist of monitoring options according to the below range of criteria
  * M27
 References: Monitoring Col AE
Valid Answers: ['No', 'Yes']
 Satisfied By: None
 Satisfies: None
Has Caveats:
      QuestionID  TargetID AnswerValue                       Note
2356         168        98         Yes  May not be representative
2357         168        99         Yes  May not be representative
2358         168       100         Yes  May not be representative
2359         168       101         Yes  May not be representative
2360         168       102         Yes  May not be representative
2361         168       103         Yes  May not be representative
2362         168       104         Yes  May not be representative
2363         168       105         Yes  May not be representative
2364         168       106         Yes  May not be represe

### Questions 49+170
My split: Q49 = A10 + R47; Q170 = M30

Correct split: Q49 = A10; Q170 = R47 + M30

Required actions:
 * reassign Q49 Control Rules caveats to Q170
 * reassign Q49 Control Rules attributes to Q170
 * Re-title Q170

In [53]:
Eq.question(49)

Question ID: 49
"Have there been operational changes in the fishery that compromise the current validity of historical data? (NB this is a different question to M30 in the Monitoring spreadsheet)"
(Category: General)
  * IF YES
  * Is the nature of fishing operations (e.g. target species, gear types, fishing locations, markets) changing?
  * Socio-economic
  * Additional requirements (not elsewhere included). These are not included in the vector matching but should pop up as warnings depending on the response.
  * Caveats
  * Y
  * Have there been operational changes in the fishery? IF YES
  * A10
  * needs to be phrased the same as the question in mo nitoring.
  * R47
  * Examples: target species, gear types, fishing locations, markets
 References: Assessment Row 89, ControlRules Row 69
Valid Answers: ['No', 'Yes']
 Satisfied By: None
 Satisfies: None
Has Caveats:
     QuestionID  TargetID AnswerValue  \
110          49         8         Yes   
111          49         9         Yes   

In [54]:
Eq.question(170)

Question ID: 170
"Is the nature of the fishery changing rapidly?"
(Category: Operational characteristics)
  * IF YES
  * CAVEATS - Second, refine the shortlist of monitoring options according to the below range of criteria
  * M30
  * be clear about time frame. There has been a big change in the past five years. What do we mean by nature? It is still a trap based fishery, but effort is increwsing
 References: Monitoring Col AH
Valid Answers: ['No', 'Yes']
 Satisfied By: None
 Satisfies: None
Has Caveats:
      QuestionID  TargetID AnswerValue                       Note
2404         170        98         Yes  May not be representative
2405         170        99         Yes  May not be representative
2406         170       100         Yes  May not be representative
2407         170       101         Yes  May not be representative
2408         170       102         Yes  May not be representative
2409         170       103         Yes  May not be representative
2410         170       104  

In [55]:
Eq._caveats.loc[(Eq._caveats['QuestionID'] == 49) & 
                (Eq._caveats['TargetID'].isin(Eq.targets_for('ControlRules'))), 'QuestionID'] = 170

In [56]:
Eq.attributes(Eq.attributes_for(49))

AttributeID ec11bea9-cbd3-3736-a89d-bbc7403e6910: IF YES
AttributeID b8a44f84-6147-38ff-9d0c-4f530bf69a8d: Is the nature of fishing operations (e.g. target species, gear types, fishing locations, markets) changing?
AttributeID 80523536-69fa-35b5-99bb-11a15baa1ffc: Socio-economic
AttributeID 12ace8bf-0d5b-3ce6-be67-5e432f992854: Additional requirements (not elsewhere included). These are not included in the vector matching but should pop up as warnings depending on the response.
AttributeID d1cb3b06-a7b0-3773-9ead-488f1c1ad64d: Caveats
AttributeID b5a7df97-3b0c-389d-b269-4ad0c038df5a: Y
AttributeID 9d430299-572b-362d-ac22-320165713cda: Have there been operational changes in the fishery? IF YES
AttributeID a8d8a83f-37e5-3b75-b22c-b9246fd76509: Have there been operational changes in the fishery that compromise the current validity of historical data? (NB this is a different question to M30 in the Monitoring spreadsheet)
AttributeID 0a9b99ec-9c5b-3537-af3c-7a1c97c37653: A10
AttributeID 54f

In [57]:
# make list of monitoring attributes that should be removed from CR question and added to M question
for x in ['b8a44f84-6147-38ff-9d0c-4f530bf69a8d', '80523536-69fa-35b5-99bb-11a15baa1ffc',
         'ec11bea9-cbd3-3736-a89d-bbc7403e6910', '185fff62-ed28-315b-865d-6c67a8548265',
         '54f4579b-1f7b-3cd6-8975-ba77a4d6b798', 'ceb4663a-e214-3c99-a3f4-1e49fedb3763',
         '3ad246fd-ffbd-314c-9f53-cb8455a2dc43']:
    move_attribute(x, 49, 170)


Removed 1 reference
Removed 1 reference
attribute map - 1 found
Removed 1 reference
Removed 1 reference
Removed 1 reference
Removed 1 reference
Removed 1 reference


In [59]:
MSSP.write_to_json(Eq.serialize(), outdir)

Creating 189 questions...
Creating 143 targets...
Creating 528 criteria...
Creating 2054 caveats...
Creating 710 attributes...
Creating 247 notes...
Creating colormap...
Output written to folder /Users/brandon/Documents/GitHub/SNAP Fisheries Spreadsheets/json/2016-06-30.


True